In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
from CA1_Yolo import *
from keras.optimizers import Adam
import time

Using TensorFlow backend.


In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [4]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

iter: 1 distances: 5179.013307955627
iter: 2 distances: 604.3330094426443
iter: 3 distances: 326.2661983757942
iter: 4 distances: 192.78979431726424
iter: 5 distances: 116.29829712576718
iter: 6 distances: 88.05611838862728
iter: 7 distances: 68.19114180508996
iter: 8 distances: 47.26887080773634
iter: 9 distances: 38.28024096838901
iter: 10 distances: 21.788784095032607
iter: 11 distances: 16.866838875484834
iter: 12 distances: 7.757274250962514
iter: 13 distances: 11.176253683007443
iter: 14 distances: 14.325222349562765
iter: 15 distances: 15.004251976055716
iter: 16 distances: 22.88884396449402
iter: 17 distances: 13.35578595470531
iter: 18 distances: 12.759565707564308
iter: 19 distances: 8.482574332176622
iter: 20 distances: 7.686161752734023
iter: 21 distances: 5.682759387782925
iter: 22 distances: 4.13569558252419
iter: 23 distances: 1.7880149433834007
iter: 24 distances: 1.554028316213501
iter: 25 distances: 1.671409580148171
iter: 26 distances: 1.5849365407984544


In [5]:
anchor_boxes

[11, 9, 26, 21, 34, 40, 43, 66, 43, 29, 62, 113, 67, 42, 84, 65, 103, 96]

In [6]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [7]:
len(train_anno)

309

In [8]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [9]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [10]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=2,
    threshold=0.5,
    max_boxes=max_boxes)







Instructions for updating:
Use `tf.cast` instead.



In [11]:
infer_mode.build((416,416))
infer_mode.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [12]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [13]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [14]:
callback = create_callbacks()

In [15]:
%%time
history = train_model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    verbose=1,
    steps_per_epoch=len(train_generator),
    epochs=300,
    callbacks=callback
)

Epoch 1/300
306/306 [==============================] - 174s 568ms/step - loss: 52.0943 - yolo_loss_layer_1_loss: 7.3452 - yolo_loss_layer_2_loss: 13.2315 - yolo_loss_layer_3_loss: 29.1895 - val_loss: 46.0383 - val_yolo_loss_layer_1_loss: 33.5750 - val_yolo_loss_layer_2_loss: 4.6725 - val_yolo_loss_layer_3_loss: 5.4620
Epoch 2/300
306/306 [==============================] - 165s 540ms/step - loss: 7.3960 - yolo_loss_layer_1_loss: 3.6690 - yolo_loss_layer_2_loss: 0.3189 - yolo_loss_layer_3_loss: 1.0804 - val_loss: 8.0138 - val_yolo_loss_layer_1_loss: 5.1456 - val_yolo_loss_layer_2_loss: 0.1271 - val_yolo_loss_layer_3_loss: 0.4156
Epoch 3/300
306/306 [==============================] - 170s 557ms/step - loss: 6.0379 - yolo_loss_layer_1_loss: 3.3541 - yolo_loss_layer_2_loss: 0.1640 - yolo_loss_layer_3_loss: 0.1973 - val_loss: 7.1612 - val_yolo_loss_layer_1_loss: 4.6495 - val_yolo_loss_layer_2_loss: 0.0905 - val_yolo_loss_layer_3_loss: 0.1023
Epoch 4/300
306/306 [=============================

In [17]:
from keras.models import model_from_json
json_string = train_model.to_json()
with open('yolo.json', 'w') as json_file:
    json_file.write(json_string)

In [18]:
train_model.save_weights('yolo_weights.h5')

In [19]:
train_model.save('yolo.h5')

In [12]:
from keras import backend as K
K.set_learning_phase(0)

In [13]:
infer_mode.load_weights('yolo_weights.h5')

In [14]:
sess = K.get_session()

In [21]:
print(infer_mode.output)

[<tf.Tensor 'conv2d_59/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>, <tf.Tensor 'conv2d_67/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>, <tf.Tensor 'conv2d_75/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>]


In [16]:
tf.train.write_graph(sess.graph_def, '', 'yolo_test.pb', as_text=False)
tf.train.write_graph(sess.graph_def, '', 'yolo_graph.pbtxt')
tf.train.Saver().save(sess, 'yolo_test.chkp')

'yolo_test.chkp'

In [17]:
from tensorflow.python.tools import freeze_graph
output_node_name = 'conv2d_59/BiasAdd,conv2d_67/BiasAdd,conv2d_75/BiasAdd'
fg = freeze_graph.freeze_graph('yolo_graph.pbtxt',
                         None, False,
                         'yolo_test.chkp',
                         output_node_name,
                         'save/restore_all',
                         'save/Const:0',
                         'yolo_test.pb',
                         True, '')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from yolo_test.chkp
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 363 variables.
INFO:tensorflow:Converted 363 variables to const ops.


In [ ]:
# from tensorflow.python.tools import optimize_for_inference_lib
# graph_def = tf.GraphDef()
# with tf.gfile.Open('yolo_test.pb', "rb") as f:
#     graph_def.ParseFromString(f.read())

# output_graph_def = optimize_for_inference_lib.optimize_for_inference(
#     graph_def, ['input_1'], [output_node_name], tf.float32.as_datatype_enum)

# with tf.gfile.GFile('yolo_test.pb', "wb") as f:
#     f.write(output_graph_def.SerializeToString())

# # Strip Const nodes.
# for i in reversed(range(len(graph_def.node))):
#     if graph_def.node[i].op == 'Const':
#         del graph_def.node[i]

In [ ]:
# tf.train.write_graph(graph_def, '', 'yolo_test.pbtxt', as_text=True)

In [22]:
net = cv2.dnn.readNetFromTensorflow('yolo_test.pb')

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_graph_simplifier.cpp:890: error: (-2:Unspecified error) Tensor's data type is not supported in function 'cv::dnn::dnn4_v20190621::getTensorContent'


In [14]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in infer_mode.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 650 variables.
INFO:tensorflow:Converted 650 variables to const ops.


In [15]:
tf.train.write_graph(frozen_graph, '', 'yolo_test1.pb', as_text=False)

'yolo_test1.pb'

In [16]:
cv2.dnn_registerLayer('YoloLossLayer', YoloLossLayer)

In [ ]:
model = cv2.dnn.readNetFromTensorflow('yolo_test1.pb')

In [ ]:
# import tensorflow as tf
# from tensorflow.tools.graph_transforms import TransformGraph
# from tensorflow.python.tools import optimize_for_inference_lib

# graph = 'yolo_test2.pb'
# with tf.gfile.FastGFile(graph,'rb') as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     tf.summary.FileWriter('logs', graph_def)

#     inp_node = 'input_1'
#     out_node = ['conv2d_59/BiasAdd','conv2d_67/BiasAdd','conv2d_75/BiasAdd']
#     graph_def = optimize_for_inference_lib.optimize_for_inference(graph_def, [inp_node], [out_node], tf.float32.as_datatype_enum)
#     graph_def = TransformGraph(graph_def, [inp_node], [out_node], ["sort_by_execution_order"])
 
#     with tf.gfile.FastGFile('frozen_inference_graph_opt.pb', 'wb') as f:
#         f.write(graph_def.SerializeToString())